In [1]:
import numpy as np 
import pandas as pd

ERROR! Session/line number was not unique in database. History logging moved to new session 323


In [2]:
df = pd.read_csv('train.csv');
test = pd.read_csv('test.csv'); 
df.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [ ]:
df.drop(columns = ['poutcome','pdays'],inplace=True); 
test.drop(columns = ['poutcome','pdays'],inplace=True); 

In [3]:
for c in df.columns : 
    if (c=='y') : continue;
    if ((df[c]=='unknown').sum()==0): continue;
    df[c] = df[c].replace('unknown',df[c].mode()[0]); 
    test[c] = test[c].replace('unknown',df[c].mode()[0]);

In [4]:
Y = df['y'].to_numpy();
df.drop(columns = ['y'],inplace=True)

In [5]:
df.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown
1,1,38,blue-collar,married,secondary,no,514,no,no,cellular,18,jun,185,1,-1,0,unknown
2,2,36,blue-collar,married,secondary,no,602,yes,no,cellular,14,may,111,2,-1,0,unknown
3,3,27,student,single,secondary,no,34,yes,no,cellular,28,may,10,2,-1,0,unknown
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown


In [6]:
Id = test['id'].to_numpy();

In [7]:
df.drop(columns = ['id'],inplace=True); 
test.drop(columns = ['id'],inplace=True);

In [8]:
df_all = pd.concat([df,test]); 

In [9]:
from sklearn.preprocessing import LabelEncoder 
for c in df.columns: 
    if (df_all[c].dtype=='O') : 
        encoder = LabelEncoder(); 
        df_all[c] = encoder.fit_transform(df_all[c]);

In [10]:
df = df_all.iloc[:df.shape[0]]
test = df_all.iloc[df.shape[0]:]

In [11]:
X = df.to_numpy()

In [12]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [13]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.1,shuffle=True); 

In [14]:
from sklearn.metrics import roc_auc_score
model1 = LGBMClassifier(class_weight={0: 1.1374, 1: 8.2742}, max_depth=20, max_leaves=15,
               n_estimators=2000, reg_alpha=0.5, reg_lambda=1, verobse=-1)
model1.fit(x_train,y_train); 
predict1 = model1.predict(x_test); 
print(roc_auc_score(y_test,predict1));

C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1036

[LightGBM] [Warning] Unknown parameter: verobse
[LightGBM] [Warning] num_leaves is set=31, max_leaves=15 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] Unknown parameter: verobse
[LightGBM] [Warning] num_leaves is set=31, max_leaves=15 will be ignored. Current value: num_leaves=31
[LightGBM] [Info] Number of positive: 81336, number of negative: 593664
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.230218 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 995
[LightGBM] [Info] Number of data points in the train set: 675000, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499168 -> initscore=-0.003327
[LightGBM] [Info] Start training from score -0.003327


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: verobse
[LightGBM] [Warning] num_leaves is set=31, max_leaves=15 will be ignored. Current value: num_leaves=31
0.9055287479195571


In [17]:
model = LGBMClassifier(verbose=-1,n_estimators=2000,class_weight={0:1.1374,1:8.2742});
#params = {'learning_rate':[0.001,0.01,0.1,0.3,1],'reg_alpha':[0,0.1,1,10],'reg_lambda':[0,0.1,1,10]}
grid = RandomizedSearchCV(n_iter=15,param_distributions=params,cv=3,scoring='roc_auc',verbose=2,estimator=model)
grid.fit(x_train,y_train);

Fitting 3 folds for each of 15 candidates, totalling 45 fits


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END learning_rate=1, max_depth=100, max_leaves=63, reg_alpha=0.1, reg_lambda=0.5; total time= 1.1min


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END learning_rate=1, max_depth=100, max_leaves=63, reg_alpha=0.1, reg_lambda=0.5; total time= 1.1min


...

[CV] END learning_rate=0.1, max_depth=5, max_leaves=0, reg_alpha=10, reg_lambda=0; total time= 1.2min


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END learning_rate=0.1, max_depth=5, max_leaves=0, reg_alpha=10, reg_lambda=0; total time= 1.3min


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END learning_rate=0.1, max_depth=5, max_leaves=0, reg_alpha=10, reg_lambda=0; total time= 1.0min


In [18]:
grid.best_score_

np.float64(0.9663346256921602)

In [21]:
from sklearn.model_selection import KFold 
models = [] 
kf= KFold(n_splits=10); 
for train_index, test_index in kf.split(X): 
    x_train,x_test,y_train,y_test = X[train_index],X[test_index],Y[train_index],Y[test_index];
    model = LGBMClassifier(class_weight={0: 1.1374, 1: 8.2742}, max_depth=30, max_leaves=0,
               n_estimators=2000, reg_alpha=10, reg_lambda=10, verbose=-1)
    model.fit(x_train,y_train); 
    predict = model.predict(x_test); 
    print(roc_auc_score(y_test,predict)); 
    models.append(model);

C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.9053676061759548


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.9090803743896299


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

0.9064833170837661


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.9085425107354467


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.9078412191111558


In [24]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.1,shuffle=True)

In [25]:
XX = [] 
for model in models : XX.append(model.predict_proba(x_train)[:,1]); 
XX = np.array(XX);

C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: U

In [27]:
XX = XX.reshape(-1,10);
XX.shape

(675000, 10)

In [30]:
XX_test = [] 
for model in models : XX_test.append(model.predict_proba(x_test)[:,1]); 
XX_test = np.array(XX_test).reshape(-1,10);

C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: U

In [31]:
model2 = LGBMClassifier(verbose=-1,n_estimators=4000,class_weight={0: 1.1374, 1: 8.2742})
model2.fit(XX,y_train); 
predict2 = model2.predict(XX_test); 


C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


NameError: name 'predit2' is not defined

In [34]:
predict2 = model2.predict_proba(XX_test)[:,1]
predict2

C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


array([0.52770498, 0.46092607, 0.44633245, ..., 0.49187524, 0.20002879,
       0.52745704], shape=(75000,))

In [35]:
print(roc_auc_score(y_test,predict2));

0.5021616768985223


In [15]:
params = {'learning_rate': [0.001, 0.01, 0.1, 0.3, 1],
         'reg_alpha': [0, 0.1,0.5, 1, 10],
         'reg_lambda': [0, 0.1,0.5, 1, 10],
          'max_depth':[0,5,10,20,30,50,100],'max_leaves':[0,7,15,63,127,255,511]
          }

In [17]:
class_weight={0:1.1374,1:8.2742}
sample_weights = np.array([class_weight[i] for i in Y]); 


In [ ]:
model = XGBClassifier(objective='multi:softmax',num_class=2,n_estimators=2000); 
grid = RandomizedSearchCV(n_iter=20,param_distributions=params,cv=3,scoring='roc_auc',verbose=2,estimator=model)
grid.fit(X,Y,**{'sample_weight':sample_weights});

Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [21]:
model = XGBClassifier(reg_lambda=0,reg_alpha=10,max_leaves=127,max_depth=0,learning_rate=0.01,
                      n_estimators=2000,objective='multi:softmax',num_class=2);

In [22]:
model.fit(x_train,y_train);


KeyboardInterrupt



In [16]:
from sklearn.metrics import roc_auc_score

In [28]:
predict = model.predict_proba(x_test)[:,1]
print(roc_auc_score(y_test,predict))

0.9891318030951646


In [29]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.08,shuffle=True);
predict = model.predict_proba(x_test)[:,1]; 
print(roc_auc_score(y_test,predict))

0.9893138009995427


In [30]:
X_test = test.to_numpy(); 
predicts = model.predict_proba(X_test)[:,1]


In [34]:
w = pd.read_csv('test.csv')

In [35]:
Id = w['id'].to_numpy()

In [36]:
sub = pd.DataFrame({'id':Id,'y':predicts});
sub.head()

,id,y
0,750000,0.043306
1,750001,0.317565
2,750002,0.005765
3,750003,0.007086
4,750004,0.037101


In [37]:
sub.to_csv('sub.csv',index=False);